In [16]:

import numpy as np
import pandas as pd   
import os
from pathlib import Path
import glob
import json
import tensorflow as tf
import tensorflow_datasets as tfds
import os
import nltk
import cv2
import matplotlib.pyplot as plt
import random
from tensorflow.keras.layers.experimental import preprocessing

print('fim')

fim


In [17]:
print(tf.__version__)
from keras.backend import manual_variable_initialization
manual_variable_initialization(True)

2.4.1


In [18]:
def GravarArquivo ( data_dict,fname):
    fname = fname +"_" +str(len(data_dict)) + '.json'
    print("gravar arquivo: " + fname + " qtd: " +  str(len(data_dict)))
    os.makedirs('Percentage_'+ str(percentageData) , exist_ok=True)
    fname = 'Percentage_'+ str(percentageData) + "/" + fname
    # Create file
    with open(fname, 'w') as outfile:
        json.dump(data_dict, outfile, ensure_ascii=False, indent=4) 
        outfile.close()

In [19]:
def plot_loss(history):
            plt.plot(history.history['loss'], label='training loss')
            plt.plot(history.history['val_loss'], label=' validation loss')
            #plt.ylim([0, 10])
            plt.xlabel('Epopch')
            plt.ylabel('Error')
            plt.legend()
            plt.grid(True)
            plt.savefig( 'Percentage_'+ str(percentageData) + '/error_datalenght.png')
            plt.show()

def plot_acc(history):  
            plt.plot(history.history['sparse_categorical_accuracy'])
            plt.plot(history.history['val_sparse_categorical_accuracy'])
            plt.title('model accuracy')
            plt.ylabel('accuracy')
            plt.xlabel('epoch')
            plt.legend(['train', 'val'], loc='upper left')
            plt.grid(True)
            plt.savefig('Percentage_'+ str(percentageData) + '/accuracy_datalenght.png')
            plt.show()

In [20]:
#Check GPU is available for training or not Or whether the tensorflow version can utilize gpu 
physical_devices = tf.config.list_physical_devices('GPU') 
print("Number of GPUs :", len(physical_devices)) 
print("Tensorflow GPU :",tf.test.is_built_with_cuda())
if len(physical_devices)>0:
    device="/GPU:0"
else:
    device="/CPU:0"
percentageData = 'fulll_material'
posTrainList=[]
posValList=[]
BATCH_SIZE=50
IMG_SIZE=(200,200)
QtdEpocasGravarCHKP = 2
Epochs = 50
lenghtDataTrain = 0
lenghtDataVal =0

Number of GPUs : 0
Tensorflow GPU : True


In [21]:
trainList=[]

with open('/home/jupyter/input/clevr-dataset/CLEVR_v1.0/questions/CLEVR_train_questions.json') as f:
    print('oi')
    data = json.load(f)
    with open('/home/jupyter/input/train_questao_material.json') as mat: 
     mat1 = json.load(mat)
     for m in mat1:
        K=int(str(m).split(':')[1].replace('''''','').replace('}',''))
        i = data['questions'][K]
       
        temp=[]
        for path in glob.glob('/home/jupyter/input/clevr-dataset/CLEVR_v1.0/images/train/'+i['image_filename']): 
            temp.append(path)
           
        temp.append(i['question'])
        temp.append(i['answer'])
        trainList.append(temp)
        posTrainList.append(K)
f.close()
labels=['Path','Question','Answer']
train_dataframe = pd.DataFrame.from_records(trainList, columns=labels)#training Dataframe 
del(data)
del(trainList)
print('fim')
lenghtDataTrain = len(posTrainList)

oi
fim


In [22]:
valList=[]
with open('/home/jupyter/input/clevr-dataset/CLEVR_v1.0/questions/CLEVR_val_questions.json') as f:
    data = json.load(f)
    with open('/home/jupyter/input/val_questao_material.json') as mat: 
     mat1 = json.load(mat)
     for m in mat1:
        K=int(str(m).split(':')[1].replace('''''','').replace('}',''))
            
        i = data['questions'][K]
        
        temp=[]
        for path in glob.glob('/home/jupyter/input/clevr-dataset/CLEVR_v1.0/images/val/'+i['image_filename']): 
            temp.append(path)
        temp.append(i['question'])
        temp.append(i['answer'])
        valList.append(temp)
        posValList.append(K)
f.close() 

val_dataframe = pd.DataFrame.from_records(valList, columns=labels)#validation Dataframe
del(data)
del(valList)
val_dataframe.head()
print('passou carregou')

passou carregou


In [23]:
vocab_set=set()#set object used to store the vocabulary

tokenizer = tfds.features.text.Tokenizer()

for i in val_dataframe['Question']:
    vocab_set.update(tokenizer.tokenize(i))
for i in train_dataframe['Question']:
    vocab_set.update(tokenizer.tokenize(i))
for i in val_dataframe['Answer']:
    vocab_set.update(tokenizer.tokenize(i))
for i in train_dataframe['Answer']:
    vocab_set.update(tokenizer.tokenize(i))
    
vocab_set.update('12aaaa')
vocab_set.update('1234sssa')

#
#Creating an Encoder and a Function to preprocess the text data during the training and inference    
    
encoder=tfds.features.text.TokenTextEncoder(vocab_set)
index=2
print("Testing the Encoder with sample questions - \n ")
example_text=encoder.encode(train_dataframe['Question'][index])
print("Original Text = "+train_dataframe['Question'][index])
print("After Encoding = "+str(example_text))


Testing the Encoder with sample questions - 
 
Original Text = What is the material of the small thing to the left of the small purple block in front of the small cyan rubber cube?
After Encoding = [8, 12, 20, 45, 85, 20, 93, 16, 44, 20, 29, 85, 20, 93, 59, 97, 40, 42, 85, 20, 93, 96, 24, 55]


In [24]:
GravarArquivo(posValList,'Val_pos' )
GravarArquivo(posTrainList,'Train_pos' )
GravarArquivo(list(vocab_set), 'Vocab_set' )



gravar arquivo: Val_pos_13520.json qtd: 13520
gravar arquivo: Train_pos_62830.json qtd: 62830
gravar arquivo: Vocab_set_99.json qtd: 99


In [25]:

#Function that uses the encoder created to encode the input question and answer string
def encode_fn(text):
    return np.array(encoder.encode(text.numpy()))


#Function to load and decode the image from the file paths in the dataframe and use the encoder function
def preprocess(ip,ans):
    img,ques=ip#ip is a list containing image paths and questions
    img=tf.io.read_file(img)
    img=tf.image.decode_jpeg(img,channels=3)
    # quantos canais de cores tem 
    img=tf.image.resize(img,IMG_SIZE)
    img=tf.math.divide(img, 255)# 
    #The question string is converted to encoded list with fixed size of 50 with padding with 0 value
    ques=tf.py_function(encode_fn,inp=[ques],Tout=tf.int32)
    paddings = [[0, 50-tf.shape(ques)[0]]]
    ques = tf.pad(ques, paddings, 'CONSTANT', constant_values=0)
    ques.set_shape([50])#Explicit shape must be defined in order to create the Input pipeline
    
    #The Answer is also encoded 
    ans=tf.py_function(encode_fn,inp=[ans],Tout=tf.int32)
    ans.set_shape([1])
    print(ans)
    return (img,ques),ans
    
def create_pipeline(dataframe):
    raw_df=tf.data.Dataset.from_tensor_slices(((dataframe['Path'],dataframe['Question']),dataframe['Answer']))
    df=raw_df.map(preprocess)#Preprocessing function is applied to the dataset
    df=df.batch(BATCH_SIZE)#The dataset is batched
    return df

#The training and validation Dataset objects are created
train_dataset=create_pipeline(train_dataframe)
validation_dataset=create_pipeline(val_dataframe)

Tensor("EagerPyFunc_1:0", shape=(1,), dtype=int32, device=/job:localhost/replica:0/task:0)
Tensor("EagerPyFunc_1:0", shape=(1,), dtype=int32, device=/job:localhost/replica:0/task:0)


In [26]:
#Creating the CNN model for image processing


CNN_Input=tf.keras.layers.Input(shape=(200,200,3),name='image_input')

mobilenetv2=tf.keras.applications.mobilenet_v2.MobileNetV2(input_shape=(200,200,3), alpha=1.0, include_top=False,
                                                      weights='imagenet', input_tensor=CNN_Input)

CNN_model=tf.keras.models.Sequential()
CNN_model.add(CNN_Input)
CNN_model.add(mobilenetv2)
CNN_model.add(tf.keras.layers.GlobalAveragePooling2D())



#Creating the RNN model for text processing
RNN_model=tf.keras.models.Sequential()

RNN_Input=tf.keras.layers.Input(shape=(50),name='text_input')
RNN_model.add(RNN_Input)
RNN_model.add(tf.keras.layers.Embedding (len(vocab_set)+1,256))
RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True,recurrent_initializer='glorot_uniform')))
RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256,return_sequences=True,recurrent_initializer='glorot_uniform')))
RNN_model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512,return_sequences=False,recurrent_initializer='glorot_uniform')))


concat=tf.keras.layers.concatenate([CNN_model.output,RNN_model.output])
dense_out=tf.keras.layers.Dense(len(vocab_set)+1,activation='softmax',name='output')(concat)

model = tf.keras.Model(inputs=[CNN_Input,RNN_Input],
                    outputs=dense_out)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])


model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_input (InputLayer)         [(None, 50)]         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 256)      25600       text_input[0][0]                 
__________________________________________________________________________________________________
image_input (InputLayer)        [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
bidirectional_3 (Bidirectional) (None, 50, 512)      1050624     embedding_1[0][0]                
____________________________________________________________________________________________

In [ ]:


def scheduler(epoch):
  if epoch < 1:
    return 0.001
  else:
    return 0.001 * tf.math.exp(0.1 * (1 - epoch))

checkpoint_path = 'Percentage_'+ str(percentageData) + '/weights-{epoch:03d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=QtdEpocasGravarCHKP*lenghtDataTrain)

print(QtdEpocasGravarCHKP*lenghtDataTrain)

LRS = tf.keras.callbacks.LearningRateScheduler(scheduler)
csv_callback=tf.keras.callbacks.CSVLogger(
   'Percentage_'+ str(percentageData) + '/'+ "Training Parameters.csv",
    separator=',', append=False
)


with tf.device(device) :
    history =  model.fit(train_dataset,
              validation_data=validation_dataset,
              callbacks=[csv_callback,cp_callback,cp_callback],
              epochs=Epochs)


hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
history.history
hist.tail()
plot_loss(history)
plot_acc(history)


125660
Epoch 1/50
 500/1257 [==========>...................] - ETA: 38:18 - loss: 0.8221 - sparse_categorical_accuracy: 0.5457
Epoch 00001: saving model to Percentage_fulll_material/weights-001.ckpt
 708/1257 [===============>..............] - ETA: 27:49 - loss: 0.7858 - sparse_categorical_accuracy: 0.5546
Epoch 00001: saving model to Percentage_fulll_material/weights-001.ckpt
 868/1257 [===================>..........] - ETA: 19:43 - loss: 0.7681 - sparse_categorical_accuracy: 0.5591
Epoch 00001: saving model to Percentage_fulll_material/weights-001.ckpt
1003/1257 [======================>.......] - ETA: 12:53 - loss: 0.7568 - sparse_categorical_accuracy: 0.5622
Epoch 00001: saving model to Percentage_fulll_material/weights-001.ckpt
1122/1257 [=========================>....] - ETA: 6:51 - loss: 0.7488 - sparse_categorical_accuracy: 0.5644
Epoch 00001: saving model to Percentage_fulll_material/weights-001.ckpt
1229/1257 [============================>.] - ETA: 1:25 - loss: 0.7427 - sparse

In [ ]:
contadorCerto =0
contadorErro =0 

for contador in range(lenghtDataTrain) :
   
    im=cv2.imread(train_dataframe.iloc[contador]['Path'])
    im=cv2.resize(im,(200,200))
    q=train_dataframe.iloc[contador]['Question'] 
    q=encoder.encode(q)
    paddings = [[0, 50-tf.shape(q)[0]]]
    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
    q=np.array(q)
    im.resize(1,200,200,3)
    q.resize(1,50)
    ans=model.predict([im,q]) 
    decodAns = encoder.decode([np.argmax(ans)])
    if train_dataframe.iloc[contador]['Answer'] != decodAns :
        contadorErro = contadorErro +1
    else:     
        contadorCerto = contadorCerto +1
      
print(contadorCerto)
print(contadorErro)
    

In [ ]:
VerifTrainList=[]
VerifTrainList.append("Acerto - " + contadorCerto )
VerifTrainList.append("Erro - " + contadorErro )
GravarArquivo(VerifTrainList,'Verif_TrainList' )


In [ ]:
contadorCerto =0
contadorErro =0 

for contador in range(lenghtDataVal) :
   
    im=cv2.imread(val_dataframe.iloc[contador]['Path'])
    im=cv2.resize(im,(200,200))
    q=val_dataframe.iloc[contador]['Question'] 
    q=encoder.encode(q)
    paddings = [[0, 50-tf.shape(q)[0]]]
    q=tf.pad(q, paddings, 'CONSTANT', constant_values=0)
    q=np.array(q)
    im.resize(1,200,200,3)
    q.resize(1,50)
    ans=model.predict([im,q]) 
    decodAns = encoder.decode([np.argmax(ans)])
    if val_dataframe.iloc[contador]['Answer'] != decodAns :
        contadorErro = contadorErro +1
    else:     
        contadorCerto = contadorCerto +1
      
print(contadorCerto)
print(contadorErro)
    

In [ ]:
VerifValList=[]
VerifValList.append("Acerto - " + contadorCerto )
VerifValList.append("Erro - " + contadorErro )
GravarArquivo(VerifValList,'Verif_ValList' )

In [ ]:
tf.keras.models.save_model(model,'Percentage_'+ str(percentageData) +  '/ModelTreinamento')

In [ ]:
import pickle

with open('Percentage_'+ str(percentageData) +  '/train_dataframe', "wb") as f:
    pickle.dump(train_dataframe, f)
with open('Percentage_'+ str(percentageData) +  '/encoder', "wb") as f:
    pickle.dump(encoder, f)



In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, show_layer_names=True)